In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [5]:
data = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/winequality-white.csv",
                 sep=",")


In [7]:
data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [21]:
train, test = train_test_split(data, test_size=.25, random_state=42)

In [23]:
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()


In [24]:
#validation data
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [25]:
signature = infer_signature(train_x, train_y)


In [38]:
#ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input(shape=(train_x.shape[1],)),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params['lr'],momentum=params['momentum']),
              loss="mean_squared_error",
              metrics=[keras.metrics.RootMeanSquaredError()]
    )

    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rsme = eval_result[1]

        #logging params and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rsme)
        #log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

    return {"loss": eval_rsme, "status": STATUS_OK, "model": model}


In [39]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x= train_x,
        train_y= train_y,
        valid_x= valid_x,
        valid_y= valid_y,
        test_x= test_x,
        test_y= test_y
    )
    return result

In [40]:
space = {
    "lr": hp.loguniform("lr",np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
    
}

In [41]:
mlflow.set_experiment("wine_quality_white")
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo= tpe.suggest,
        max_evals=4,
        trials=trials
    )

    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]

    mlflow.log_params(best)
    mlflow.log_metric("Evalrmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], 'model', signature=signature)

    print(f"best params: {best}")
    print(f"best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 41s 925ms/step - loss: 36.2126 - root_mean_squared_error: 5.9792
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.8190 - root_mean_squared_error: 5.9848
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 35.7520 - root_mean_squared_error: 5.9792
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 35.7369 - root_mean_squared_error: 5.9780
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 35.5884 - root_mean_squared_error: 5.9511   
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 34.9390 - root_mean_squared_error: 5.9103
 9/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 34.6620 - root_mean_squared_error: 5.8610
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 34.3668 - root_mean_squared_error: 5.8610
11/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 34.0852 - root_mean_squared_error: 5.8365
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 33.8048 - root_mean_squared_error: 5.8120
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 33